# Продвинутый Python, лекция 4

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Plotly

Вернемся к визуализации. Теперь хотим интерактивности, да и вообще чего-то более мощного - то, чего не хватает двум прошлым библиотекам

Plotly - это основа из seaborn (то есть matplotlib в квадрате) + навешенный фронтенд на JS

Вместе с Flask существует отдельная библиотека Dash дял создания красивых и приятных дашбордов (будем говорить на лекции-семинаре про Flask)

[Документация](https://plotly.com/python/)

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd

Начнем с базы - отрисовка самых простых графиков:

* line - линия

* scatter - точки

In [ ]:
x = np.linspace(0, 1, num=100, endpoint=True)
y = np.sin(2 * x) + np.cos(9 * x)

px.scatter(x=x, y=y).show()

Ух ты, уже достаточно прикольно (видим значения, получаем интерактивность)

Давайте на базовом уровне проговорим о прорисовке внутри Plotly. Прорисовка состоит из создания словарика:

```
import plotly.io as pio

fig = dict({
    "data": [{"type": "bar",
              "x": [1, 2, 3],
              "y": [1, 3, 2]}],
    "layout": {"title": {"text": "A Figure Specified By Python Dictionary"}}
})

pio.show(fig)
```

По сути мы создаем большой вложенный словарь из инструкций, как и что прорисовывать. Понятное дело, что никто этим не хочет заниматься, поэтому для этого есть структура фигуры, внутри которой создается этот самый словарик

Чуть получше - создать фигуру, на которую будем все переносить (ну как в matplotlib)

И добавим сразу 2 линии!

In [ ]:
fig = go.Figure() # Создали фигуру, внутри которой будем добавлять отрисовку
fig.add_trace(go.Scatter(x=x, y=y, name="$y = \sin 2x + \cos 9x$"))
fig.add_trace(go.Scatter(x=x, y=x, name="$y = x$"))
fig.show() #Хоба, стала линия, да еще и легенду сам добавил
# Но видим фигню при наведении, исправить можно, если писать в HTML-стиле все это, а не в LaTEX

Не нравится, что легенда сбоку, хотим ее сдвинуть, давайте сдвинем!

Отдельная функция: [update_layout](https://plotly.com/python/reference/layout/). Аргументов и значений там примерно сотня, учить их все, мы, конечно же, не будем

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name="y = sin 2x + cos 9x"))
fig.add_trace(go.Scatter(x=x, y=x*x, name="y = x<sup>2</sup>"))
fig.update_layout(legend_orientation="h", 
                  margin=dict(l=0, r=0, t=0, b=0), #есть еще отступы, по дефолту 20 пикселей. Можем их убрать
                  legend=dict(x=.5, xanchor="center")) #а еще отцентрировать легенду
# l - left, r - right, t - top, b - bottom
fig.show()

Окей, по классике надо добавить подписи к осям и название. Тоже делаем через update_layout:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name="y = sin 2x + cos 9x"))
fig.add_trace(go.Scatter(x=x, y=x*x, name="y = x<sup>2</sup>"))
fig.update_layout(legend_orientation="h", 
                  margin=dict(l=0, r=20, t=30, b=0), #надо сделать сдвиг вверху, чтобы название влезло. Сдвиг считается от самого графика
                  legend=dict(x=.5, xanchor="center"))
fig.update_layout(title="Functions",
                  xaxis_title="x",
                  yaxis_title="y")
fig.show()

Но все-таки хотим точки, а не линию. Решение есть - это mode внутри отрисовки!

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name="y = sin 2x + cos 9x"))
fig.add_trace(go.Scatter(x=x, y=x*x, name="y = x<sup>2</sup>", mode='markers'))
fig.update_layout(legend_orientation="h", 
                  margin=dict(l=0, r=20, t=30, b=0),
                  legend=dict(x=.5, xanchor="center"),
                  hovermode='x')
fig.update_layout(title="Functions",
                  xaxis_title="x",
                  yaxis_title="y")
fig.update_traces(hoverinfo="all", hovertemplate="Аргумент: %{x}<br>Функция: %{y}") #Обновляем то, как показывать значения внутри
fig.show()

Не будем дальше останавливаться на возможности модификации самих линий и их отрисовок (это можно посмотреть [здесь](https://plotly.com/python/marker-style/))

Не будем останавливаться, просто потому что их невероятное количество внутри plotly. Просто приведу пример:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, name="y = sin 2x + cos 9x", line=dict(color='Red', width=5)))
fig.add_trace(go.Scatter(x=x, y=x*x, name="y = x<sup>2</sup>", mode="markers", marker=dict(color='LightSkyBlue', size=6)))
# Можно задать отдельно для точек (с помощью marker - словарь, где указываем параметры)
# Отдельно для линий: line
fig.update_layout(legend_orientation="h", 
                  margin=dict(l=0, r=20, t=30, b=0), #надо сделать сдвиг вверху, чтобы название влезло. Сдвиг считается от самого графика
                  legend=dict(x=.5, xanchor="center"))
fig.update_layout(title="Functions",
                  xaxis_title="x",
                  yaxis_title="y",
                  hovermode='x')
fig.update_traces(hoverinfo="all", hovertemplate="Аргумент: %{x}<br>Функция: %{y}")
fig.show()

Давайте попробуем создать несколько графиков:

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Plot 1", "Plot 2"]) # делается просто с помощью make_subplots

fig.add_trace(go.Scatter(x=x, y=y,  name='y = sin 2x + cos 9x'), 1, 1) #в конце указываем для add_trace куда добавляем (нумерация с 1)
fig.add_trace(go.Scatter(x=x, y=x * x,  name='y = x<sup>2</sup>'), 1, 2)
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="Аргумент: %{x}<br>Функция: %{y}")

fig.update_layout(title="Plot Title", title_x=0.5)
fig.update_xaxes(title='Ось X графика 1', col=1, row=1) # отдельно как подписать оси в каждом случае
fig.update_xaxes(title='Ось X графика 2', col=2, row=1)
fig.update_yaxes(title='Ось Y графика 1', col=1, row=1)
fig.update_yaxes(title='Ось Y графика 2', col=2, row=1)

fig.show()

А теперь хотим создать три графика, где два будут в первой половине, а третий - во второй половине (как мы уже делали в matplotlib)

In [ ]:
fig = make_subplots(rows=2, cols=2, column_widths=[1, 2], # можно задать отношение колонок
                    specs=[
                        [{}, {"rowspan": 2}], #{} - занимай клетку, None - нет графика в клетке
                        [{}, {}] # rowspan - объединяем несколько строк в одну, colspan - объединяем несколько колонок в одну
                    ]
)

# для объединения по колонкам используем colspan (идет все прямиком из HTML)

fig.add_trace(go.Scatter(x=x, y=y, name='y = sin 2x + cos 9x'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=x * x,  name='y = x<sup>2</sup>'), 1, 2)
fig.add_trace(go.Scatter(x=x, y=y, name='y = sin 2x + cos 9x'), 2, 1)
fig.add_trace(go.Scatter(x=x, y=y, name='y = sin 2x + cos 9x'), 2, 2)

fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=0, b=0),
                  width=800, # Задаем размеры самой фигуры
                  height=1000)
fig.update_traces(hoverinfo="all", hovertemplate="Аргумент: %{x}<br>Функция: %{y}")
fig.show()

Давайте посмотрим на примеры еще базовых графиков (а более интересные вещи будут на семинаре, в том числе и анимация)

Для этого создадим датасет, состоящий из подбрасывания кубиков и будем его использовать:

In [ ]:
dices = pd.DataFrame(np.random.randint(low=1, high=7, size=(100, 2)), columns=('f', 's'))


dices['overall'] = dices.f + dices.s

sum_counts = dices.overall.value_counts().sort_index()
dices.head()

,f,s,overall
0,6,1,7
1,5,6,11
2,4,1,5
3,3,6,9
4,1,1,2


In [ ]:
sum_counts.head()

2     4
3     3
4     6
5    14
6     9
Name: overall, dtype: int64

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(values=sum_counts, labels=sum_counts.index))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(values=sum_counts, labels=sum_counts.index, sort = False))
fig.show()

In [ ]:
fig = go.Figure()
pull = [0]*len(sum_counts)
pull[sum_counts.tolist().index(sum_counts.max())] = 0.2
fig.add_trace(go.Pie(values=sum_counts, labels=sum_counts.index, pull=pull, hole=0.7))

fig.update_layout(
    annotations=[dict(text='Суммы очков<br>при броске<br>2 игральных костей', x=0.5, y=0.5, font_size=14, showarrow=False)])
fig.show()

In [ ]:
labels = ["Overall actions: " + str(sum(sum_counts))]
parents = [""]
values = [sum(sum_counts)]

second_level_dict = {x:'Actions: ' + str(sum_counts[x]) + '<br>Σ = ' + str(x) for x in sum_counts.index}
labels += map(lambda x: second_level_dict[x], sum_counts.index)
parents += [labels[0]]*len(sum_counts)
values += sum_counts.tolist()

third_level = dices.groupby(['f', 's']).count().reset_index()
third_level.rename(columns={'overall':'Value'}, inplace=True)
third_level['overall'] = third_level['f'] + third_level['s']
third_level['Label'] = third_level['f'].map(str) + ' + ' + third_level['s'].map(str)
third_level['Parent'] = third_level['overall'].map(lambda x: second_level_dict[x])
values += third_level['Value'].tolist()
parents += third_level['Parent'].tolist()
labels += third_level['Label'].tolist()

fig = go.Figure(go.Sunburst(
    labels = labels,
    parents = parents,
    values=values,
    branchvalues="total"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

In [ ]:
print(labels)
print(parents)
print(values)

['Overall actions: 100', 'Actions: 3<br>Σ = 2', 'Actions: 11<br>Σ = 3', 'Actions: 3<br>Σ = 4', 'Actions: 12<br>Σ = 5', 'Actions: 8<br>Σ = 6', 'Actions: 14<br>Σ = 7', 'Actions: 12<br>Σ = 8', 'Actions: 19<br>Σ = 9', 'Actions: 6<br>Σ = 10', 'Actions: 8<br>Σ = 11', 'Actions: 4<br>Σ = 12', '1 + 1', '1 + 2', '1 + 4', '1 + 5', '1 + 6', '2 + 1', '2 + 2', '2 + 3', '2 + 5', '2 + 6', '3 + 2', '3 + 3', '3 + 4', '3 + 5', '3 + 6', '4 + 1', '4 + 3', '4 + 4', '4 + 5', '5 + 1', '5 + 2', '5 + 3', '5 + 4', '5 + 5', '5 + 6', '6 + 1', '6 + 2', '6 + 3', '6 + 4', '6 + 5', '6 + 6']
['', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Overall actions: 100', 'Actions: 3<br>Σ = 2', 'Actions: 11<br>Σ = 3', 'Actions: 12<br>Σ = 5', 'Actions: 8<br>Σ = 6', 'Actions: 14<br>Σ = 7', 'Actions: 11<br>Σ = 3', 'Actions: 3<br>Σ = 4', '

Аналогично есть:

* Bar - столбчатая диаграмма

* Histogram - гистограмма

* Box - ящик с усами

* Violin - диаграмма виолончель

С ними поиграемся на семинаре)

In [ ]:
from scipy.stats import norm
r = norm.rvs(size=1000)

x_norm = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)

fig = go.Figure()
fig.add_trace(go.Histogram(x=r, name='"Экспериментальные" данные'))
fig.update_layout(
    title="Пример гистограммы на основе нормального распределения",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(nbinsx=200, #добавим большое число бинов
                           x=r, histnorm='probability density', name='"Экспериментальные" данные'))
fig.add_trace(go.Scatter(x=x_norm, y=norm.pdf(x_norm), name='Теоретическая форма нормального распределения'))
fig.update_layout(
    title="Пример гистограммы на основе нормального распределения",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))

fig.show()

Давайте просто возьмем и развернем все это добро (делается абсолютно также, как и в случае с seaborn)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(nbinsx=200, #добавим большое число бинов
                           y=r, histnorm='probability density', name='"Экспериментальные" данные'))
fig.add_trace(go.Scatter(y=x_norm, x=norm.pdf(x_norm), name='Теоретическая форма нормального распределения'))
fig.update_layout(
    title="Пример гистограммы на основе нормального распределения",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))

fig.show()

И на самом деле даже больше - можно работать с картами!

In [ ]:
cities = pd.read_csv('https://raw.githubusercontent.com/hflabs/city/master/city.csv')
cities.head()

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,fias_level,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year
0,"Респ Адыгея, г Адыгейск",385200.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Адыгейск,...,4,0,79403000000,79703000001,107,UTC+3,44.878414,39.190289,12689,1969
1,г Майкоп,385000.0,Россия,Южный,Респ,Адыгея,NaN,NaN,г,Майкоп,...,4,2,79401000000,79701000001,105,UTC+3,44.609827,40.100661,144055,1857
2,г Горно-Алтайск,649000.0,Россия,Сибирский,Респ,Алтай,NaN,NaN,г,Горно-Алтайск,...,4,2,84401000000,84701000001,400,UTC+7,51.958103,85.960324,62861,1830
3,"Алтайский край, г Алейск",658125.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Алейск,...,4,0,1403000000,1703000001,2201,UTC+7,52.492251,82.779361,28528,1913
4,г Барнаул,656000.0,Россия,Сибирский,край,Алтайский,NaN,NaN,г,Барнаул,...,4,2,1401000000,1701000001,2200,UTC+7,53.347997,83.779806,635585,1730


In [ ]:
fig.update_layout(mapbox_style="open-street-map")

fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'], lon=cities['geo_lon']))
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'], lon=cities['geo_lon']))
capital = cities[cities['region']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

Чего-то не хватает, да? Наверное, как минимум, названий городов!

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'], lon=cities['geo_lon'], text=cities['city']))
capital = cities[cities['region']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

И давайте в заключение просто нарисуем линию между Мск и Питером (потому что почему бы и нет)

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'], lon=cities['geo_lon'], text=cities['city']))
capital = cities[cities['region']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])
fig.add_trace(go.Scattermapbox(mode = "lines",
                               hoverinfo='skip',
                               lat=cities[cities['region'].isin(['Санкт-Петербург', 'Москва'])]['geo_lat'],
                               lon=cities[cities['region'].isin(['Санкт-Петербург', 'Москва'])]['geo_lon']))
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

## Typing

За оставшееся время давайте обсудим одну вещь, которая важна с точки зрения читабельности кода (да и в целом делает работу проще)

Это Typing. А что это?

Ну, давайте с вами вспомним языки, которые мы знаем: C++, Pascal, C (возможно, что-нибудь еще). Что у них есть такого, чего нет у Python?

А у них при объявлении переменных мы обозначаем тип! Одно из отличий Python - это поддержка **динамической типизации** (когда у C, C++ etc **статическая типизация**)

Что это обозначает? Это обозначает, что в питоне при объявлении переменных их тип определяется в тот момент, когда он принимает значение (в других языках - мы вначале объявляем переменную и ее тип, а только потом делаем присваивание. Почему так? Потому что в реальности у переименных вообще нет типа (когда мы вызываем type(), он берет значение из переменной и по значению определяет, что происходит)



### Окей, а в чем проблема-то? Есть типизация и есть, чего прикопался?

Добавление типической типизации дает возможность решить 2 проблемы:

1. Сделать ваш код быстрее (за счет того, что не надо постоянно проверять тип)

2. Сделать ваш код более читабельным и безопасным 

Если вы попробуете передать строку, а не число в качестве аргумента, то с типизацией он не даст это сделать. И тем более представьте, если у вас огромный код, в котором вам надо посмотреть, что же там за тип у вот этой  переменной (а она то ли строка, то ли словарь, то ли вообще инстанс кастомного класса из другого модуля), а с помощью таких определений вы сразу знаете, что произошло

In [ ]:
def m(a):
    res = 0
    for i in a:
        res += i
    return res

In [ ]:
from typing import List

def m_t(a : List[int]) -> int:
    res : int
    res = 0
    for i in a:
        res += i
    return res

In [ ]:
import numpy as np 

k = list(np.arange(0, 100000))

In [ ]:
%%time

m(k)

CPU times: user 14.4 ms, sys: 0 ns, total: 14.4 ms
Wall time: 14.6 ms


4999950000

In [ ]:
%%time

m_t(k)

CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 11.8 ms


4999950000

### Лааааадно, убедил, давай сюда свою типизацию

Ураааа, ну, поехали тогда)

Библиотека, отвечающая за типизацию - это [typing](https://docs.python.org/3/library/typing.html). Библиотека встроенная, устанавливать дополнительно не надо

Базово, любые объявления идут через двоеточия, вывод функции через ->:

In [ ]:
n : int = 5 #объявил переменную n с типом integer и значением 5
n = "12"
type(n)

str

In [ ]:
def red(k:int, s:str) -> str: #задали типы для наших переменных и какой вывод
    return str(k) + s

In [ ]:
red(k=4.05, s="12")

'4.0512'

У нас каждый раз получается обманывать систему и передавать не то, что нужно, как так...

Ну дело в том, что структура Python никак не изменилась из-за того, что вы написали. Во многом typing - это полезная штука для информации (иногда для оптимизации кода), но не более того. Он не делает статическую типизацию (такой в Python не предусмотрено)

Что есть?

*   Optional[type] - если хотим указать, что может быть либо инстанс типа или же None
*   Any - может быть что угодно
*   Union[type_1, type_2] - если хотим указать, что инстанс может быть каким-то из нескольких вариантов (например, Union[int, float])



In [ ]:
from typing import Optional, Union, Any

def dummy(x: Union[int, float]) -> Union[int, float]: # может быть как int, так и float
    return x + 10

def opt(x: Optional[int]) -> str: #может быть int, а может быть None
    if x is None:
        return "None"
    return str(x)

def any(x: Any) -> None: # Может быть что угодно
    return None

Посложнее:

In [ ]:
from typing import List, Tuple, Dict, Set

words: List = ["hello", "world"]
bauthor: Tuple[str, str] = ("Book", "Author")
book_authors: Dict[str, str] = {"Fahrenheit 451": "Bradbury"}
numb : Set[int] = {1,2,3,4,5,6}

## Аналогично есть с
## FrozenSet
## DefaultDict, OrderedDict

Давайте чуть-чуть про возврат у функций (напоминаю, что хороший тон требует, чтобы вывод у функции везде должен быть одного типа):

* None - если ничего не выводим

* NoReturn - если функция не кончается (например, while True)

* Iterable[T] - возвращаем генератор (привет yield)

In [ ]:
from typing import Iterable

def generate_two() -> Iterable[int]:
    yield 1
    yield 2

Если необходимо вызвать функцию:

In [ ]:
from typing import Callable

def help() -> None:
    print("This is help string")

def render_hundreds(num: int) -> str:
    return str(num // 100)

def app(helper: Callable[[], None], renderer: Callable[[int], str]):
    helper()
    num = 12345
    print(renderer(num))

app(help, render_hundreds)

This is help string
123


ОТдельно создать переменную дженерика

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class LinkedList(Generic[T]):
    data: T
    next: "LinkedList[T]"

    def __init__(self, data: T):
        self.data = data

## Попугай дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Kea.jpg/1024px-Kea.jpg)

Это кеа, или же клоуны гор

Их так назвали из-за их особенной сообразительности (они умеют решать загадки, например) и любознательности. Они не упускают случая исследовать содержимое рюкзаков или автомобилей, которые им приглянулись, чем портят вещи, машины, да что угодно

Но при этом они очень хорошо и быстро привыкают к людям и в целов очень игривые :з